# Obtain data from Wikipage - Part 1

In [2]:
import requests
import pandas as pd

# Load data from Wikipage

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)

type(df_list)

df = df_list[0]

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
df.shape

(288, 3)

In [4]:
# Check how many Borough have not assigned: 77
df.groupby('Borough').count()

,Postcode,Neighbourhood
Borough,,
Central Toronto,17,17
Downtown Toronto,37,37
East Toronto,7,7
East York,6,6
Etobicoke,45,45
Mississauga,1,1
North York,38,38
Not assigned,77,77
Queen's Park,1,1


In [5]:
# Ignore the cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
df.shape
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [6]:
# Looks good - 77 less rows :-)
#df.loc[df['Postcode'] == 'M4B']
df.shape

(211, 3)

In [7]:
# Prepare result dataframe df_result

df_result = df.copy()
df_result['Neighbourhood']=''

# Drop dublicates - only one row per Postcode
df_result.drop_duplicates(keep='first', inplace=True)

#df_result.loc[df_result['Postcode'] == 'M4B']

df_result.reset_index(drop=True, inplace=True)

df_result.head()
df_result.shape

(103, 3)

In [8]:
#df_result.at[0,'Borough']
#df_result.loc[df_result['Postcode'] == 'M4B']

In [9]:
# Update Neigbourhood colum of result dataframe df_result per postcode and get all Neigbourhoods in one string (value)

for index_r, row_r in df_result.iterrows():
        #print('S_r:',index_r, row_r.Postcode, row_r.Neighbourhood)
        value=''
        for index, row in df.iterrows():
            #print(row.Postcode, row.Neighbourhood)
            if row_r.Postcode==row.Postcode:
                if value=='':
                    value = str(row.Neighbourhood)
                else:
                    value = value +', ' + str(row.Neighbourhood)
                #print('chacka', value)
            #print('_:', row.Neighbourhood)   
        df_result.at[index_r,'Neighbourhood']=value
        #print('E_r:',index_r,df_result.at[index_r,'Neighbourhood'])
        

In [10]:
# Check the results
df_result.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [11]:
# Check the results
pd.options.display.max_colwidth = 200
df_result.loc[df_result['Postcode'] == 'M5A']

,Postcode,Borough,Neighbourhood
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [12]:
# Look on all Neigbourhoods with 'Not assigned'
df_result.loc[df_result['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood
4,M7A,Queen's Park,Not assigned


In [13]:
# Update Neighbourhood with Queen's Park for this row
df_result.at[4,'Neighbourhood']="Queen's Park"

In [14]:
# Check the result
df_result.loc[df_result['Neighbourhood'] == "Queen's Park"]

,Postcode,Borough,Neighbourhood
4,M7A,Queen's Park,Queen's Park


In [15]:
df_result.shape

(103, 3)

# Get the latitude and the longitude coordinates of each neighborhood - Part 2

In [46]:
# Load csv with Geocodes

df_geocodes = pd.read_csv('https://cocl.us/Geospatial_data')
df_geocodes.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [47]:

# Change Kewy of df_result & df_geocodes
df_result.rename(columns={"Postcode": "PostalCode"}, inplace=True)

df_geocodes.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

In [52]:
# Merge df_result with df_geocodes - This is the final result
 
df_final_result = pd.merge(df_result, df_geocodes, on='PostalCode')
df_final_result.head(15)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [51]:
# Check Results - looks good :-)
df_final_result.loc[df_result['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [53]:
df_final_result.shape

(103, 5)